<img src="ev_cars.jpg" style="width:300px; float: right; margin: 0 40px 40px 40px;">Baigiamasis projektas</img>


# Elektromobilių statistikos analizė. Automobilių registracijų kiekio prognozė pagal metus ir mėnesį.  Lietuva


In [1]:
# Bibliotekų importavimas
import numpy as np
import pandas as pd

# VSCode nustatymai
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

### Funkcijos

In [2]:
def eiluciu_trinimas(tekstas, stulp, df_f):
    eilutes_trinimui = df_f[df_f[stulp] == tekstas]
    df_f = df_f.drop(eilutes_trinimui.index)
    return df_f


### Atvirai prieinamų duomenų failų nuskaitymas:

In [3]:
# stulpelių pavadinimų ir jų aprašymų nuskaitymas iš xls tipo failo
stulp_pavad = ['stulpelio pavadinimas', 'Aprasymas']
df_xls = pd.read_excel('Metaduomenys v_4.xls', sheet_name='Metadata_TP_parko_duomenys', usecols='B:C', skiprows=4, names = stulp_pavad)
df_xls

,stulpelio pavadinimas,Aprasymas
0,KOMERCINIS_PAV,"Komercinis pavadinimas (kai VALD_TIPAS - ""Fizinis"" ir MARKE+KOMERCINIS_PAV+SAVIVALDYBE=1, tomet vietoje ""Komercinis pavadinimas"" yra nurodoma ""Nuasmeninta"", atsižvelgiant į asmens duomenų apsaugą)"
1,GAMINTOJO_PAV,Gamintojo pavadinimas
2,GAMINTOJO_PAV_BAZ,Bazinės transporto priemonės gamintojo pavadinimas
3,TIPAS,"Tipas (kai VALD_TIPAS - ""Fizinis"" ir MARKE+KOMERCINIS_PAV+SAVIVALDYBE=1, tomet vietoje ""Tipas"" yra nurodoma ""Nuasmeninta"", atsižvelgiant į asmens duomenų apsaugą)"
4,VARIANTAS,"Variantas (kai VALD_TIPAS - ""Fizinis"" ir MARKE+KOMERCINIS_PAV+SAVIVALDYBE=1, tomet vietoje ""Variantas"" yra nurodoma ""Nuasmeninta"", atsižvelgiant į asmens duomenų apsaugą)"
5,VERSIJA,"Versija (kai VALD_TIPAS - ""Fizinis"" ir MARKE+KOMERCINIS_PAV+SAVIVALDYBE=1, tomet vietoje ""Versija"" yra nurodoma ""Nuasmeninta"", atsižvelgiant į asmens duomenų apsaugą)"
6,ES_TIPO_PATVIRTINIMO_NR,"ES tipo patvirtinimo numeris (kai MARKE+KOMERCINIS_PAV+SAVIVALDYBE+KATEGORIJA_KLASE (kai reikšmė M1;N1))=1 (lyginant visus įregistruotus registro objektus), tomet vietoje ""Tipo patvirtinimo numeris"" yra nurodoma ""Nuasmeninta"", atsižvelgiant į asmens duomenų apsaugą.)"
7,NAC_TIPO_PATVIRTINIMO_NR,"Nacionalinis tipo patvirtinimo, įskaitant išplėtimą, numeris (kai VALD_TIPAS - ""Fizinis"" ir MARKE+KOMERCINIS_PAV+SAVIVALDYBE=1, tomet vietoje ""Nacionalinis tipo patvirtinimo, įskaitant išplėtimą, numeris"" yra nurodoma ""Nuasmeninta"", atsižvelgiant į asmens duomenų apsaugą)"
8,INVIDUAL_PATVIRTINIMO_NR,"Individualaus patvirtinimo numeris (kai VALD_TIPAS - ""Fizinis"" ir MARKE+KOMERCINIS_PAV+SAVIVALDYBE=1, tomet vietoje ""Individualaus patvirtinimo numeris"" yra nurodoma ""Nuasmeninta"", atsižvelgiant į asmens duomenų apsaugą)"
9,INTERPOLIACIJA,"Interpoliacijos šeimos identifikatorius Kai VALD_TIPAS - ""Fizinis"" ir MARKE+KOMERCINIS_PAV+SAVIVALDYBE=1, tomet vietoje ""Interpoliacijos šeimos identifikatorius"" yra nurodoma ""Nuasmeninta"", atsižvelgiant į asmens duomenų apsaugą)"


In [4]:
# Duomenų nuskaitymas iš pagrindinio duomenų failo
# Pradžiai pasirenkami stulpeliai:
# MARKE, KOMERCINIS_PAV, ELEKTRINE_TP, PIRM_REG_DATA_LT, GALIOJIMO_LAIK, APSKRITIS

# Duomenų failas atsisiunčiamas iš https://www.regitra.lt/lt/paslaugos/duomenu-teikimas/atviri-duomenys-1/transporto-priemones-3

df = pd.read_csv('Atviri_TP_parko_duomenys.csv', usecols=['MARKE','KOMERCINIS_PAV', 'PIRM_REG_DATA_LT', 'APSKRITIS','ELEKTRINE_TP', 'GALIOJIMO_LAIK', 'GAMYBOS_METAI']) 
print('eilučių skaičius: ',len(df)) # eilučių skaičius
df_kuras = df
df = df.loc[df['ELEKTRINE_TP'] == 'Y']
df = df.reset_index(drop=True)
df

eilučių skaičius:  2274514


,MARKE,KOMERCINIS_PAV,ELEKTRINE_TP,GAMYBOS_METAI,PIRM_REG_DATA_LT,GALIOJIMO_LAIK,APSKRITIS
0,NISSAN,LEAF,Y,NaN,2017-04-29,NaN,KAU
1,TESLA MOTORS,MODEL X,Y,NaN,2017-05-19,NaN,KAU
2,Nuasmeninta,Nuasmeninta,Y,NaN,2017-07-10,NaN,KAU
3,NISSAN,LEAF,Y,NaN,2017-09-08,NaN,TEL
4,NISSAN,NISSAN LEAF 40KWH,Y,NaN,2018-11-30,NaN,KAU
...,...,...,...,...,...,...,...
16065,MAZDA,MAZDA MX-30,Y,NaN,2024-04-04,NaN,KAU
16066,MAZDA,MAZDA MX-30,Y,NaN,2024-04-11,NaN,VIL
16067,GREENMO,-,Y,NaN,2024-04-23,NaN,KAU
16068,GREENMO,-,Y,NaN,2024-04-27,NaN,KAU


### Duomenų apžvalda

In [5]:
# Informacija apie df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16070 entries, 0 to 16069
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MARKE             16070 non-null  object 
 1   KOMERCINIS_PAV    16062 non-null  object 
 2   ELEKTRINE_TP      16070 non-null  object 
 3   GAMYBOS_METAI     6 non-null      float64
 4   PIRM_REG_DATA_LT  16060 non-null  object 
 5   GALIOJIMO_LAIK    689 non-null    object 
 6   APSKRITIS         16041 non-null  object 
dtypes: float64(1), object(6)
memory usage: 879.0+ KB


In [6]:
# Tikrinam kas tas 'GALIOJIMO_LAIK
df[df['GALIOJIMO_LAIK'] == np.nan ]

,MARKE,KOMERCINIS_PAV,ELEKTRINE_TP,GAMYBOS_METAI,PIRM_REG_DATA_LT,GALIOJIMO_LAIK,APSKRITIS


In [7]:
df[df['GALIOJIMO_LAIK'] != np.nan ]

,MARKE,KOMERCINIS_PAV,ELEKTRINE_TP,GAMYBOS_METAI,PIRM_REG_DATA_LT,GALIOJIMO_LAIK,APSKRITIS
0,NISSAN,LEAF,Y,NaN,2017-04-29,NaN,KAU
1,TESLA MOTORS,MODEL X,Y,NaN,2017-05-19,NaN,KAU
2,Nuasmeninta,Nuasmeninta,Y,NaN,2017-07-10,NaN,KAU
3,NISSAN,LEAF,Y,NaN,2017-09-08,NaN,TEL
4,NISSAN,NISSAN LEAF 40KWH,Y,NaN,2018-11-30,NaN,KAU
...,...,...,...,...,...,...,...
16065,MAZDA,MAZDA MX-30,Y,NaN,2024-04-04,NaN,KAU
16066,MAZDA,MAZDA MX-30,Y,NaN,2024-04-11,NaN,VIL
16067,GREENMO,-,Y,NaN,2024-04-23,NaN,KAU
16068,GREENMO,-,Y,NaN,2024-04-27,NaN,KAU


In [8]:
type(df['GALIOJIMO_LAIK'][0])

float

In [9]:
df['GALIOJIMO_LAIK']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
16065    NaN
16066    NaN
16067    NaN
16068    NaN
16069    NaN
Name: GALIOJIMO_LAIK, Length: 16070, dtype: object

In [10]:

df['GALIOJIMO_LAIK'] = df['GALIOJIMO_LAIK'].fillna('2025')
df

,MARKE,KOMERCINIS_PAV,ELEKTRINE_TP,GAMYBOS_METAI,PIRM_REG_DATA_LT,GALIOJIMO_LAIK,APSKRITIS
0,NISSAN,LEAF,Y,NaN,2017-04-29,2025,KAU
1,TESLA MOTORS,MODEL X,Y,NaN,2017-05-19,2025,KAU
2,Nuasmeninta,Nuasmeninta,Y,NaN,2017-07-10,2025,KAU
3,NISSAN,LEAF,Y,NaN,2017-09-08,2025,TEL
4,NISSAN,NISSAN LEAF 40KWH,Y,NaN,2018-11-30,2025,KAU
...,...,...,...,...,...,...,...
16065,MAZDA,MAZDA MX-30,Y,NaN,2024-04-04,2025,KAU
16066,MAZDA,MAZDA MX-30,Y,NaN,2024-04-11,2025,VIL
16067,GREENMO,-,Y,NaN,2024-04-23,2025,KAU
16068,GREENMO,-,Y,NaN,2024-04-27,2025,KAU


In [11]:

df[df['GALIOJIMO_LAIK'] != '2025']

,MARKE,KOMERCINIS_PAV,ELEKTRINE_TP,GAMYBOS_METAI,PIRM_REG_DATA_LT,GALIOJIMO_LAIK,APSKRITIS
37,HYUNDAI,KONA. KAUAI,Y,NaN,2020-06-17,2024-08-30,ŠIA
44,NISSAN,NISSAN LEAF 40KWH,Y,NaN,2020-05-20,2027-10-15,VIL
50,RENAULT,ZOE,Y,NaN,2020-07-07,2025-09-30,VIL
58,Nuasmeninta,Nuasmeninta,Y,NaN,2020-11-07,2026-02-05,ŠIA
62,RENAULT,ZOE,Y,NaN,2020-08-28,2025-12-01,VIL
...,...,...,...,...,...,...,...
15959,NISSAN,NISSAN LEAF 40KWH,Y,NaN,2024-06-11,2024-07-11,NaN
15960,MERCEDES-BENZ,EQE 350 4MATIC,Y,NaN,2024-06-12,2024-07-12,KAU
16015,TESLA MOTORS,Model X,Y,NaN,2024-06-27,2024-07-27,TEL
16040,VOLKSWAGEN,GOLF,Y,NaN,2024-06-05,2024-07-05,NaN


### Duomenu valymas:

In [12]:
# Nan reikšmių tikrinimas
df.isnull().any()

MARKE               False
KOMERCINIS_PAV       True
ELEKTRINE_TP        False
GAMYBOS_METAI        True
PIRM_REG_DATA_LT     True
GALIOJIMO_LAIK      False
APSKRITIS            True
dtype: bool

In [13]:
# Nan reikšmių kiekiai
df.isnull().sum()

MARKE                   0
KOMERCINIS_PAV          8
ELEKTRINE_TP            0
GAMYBOS_METAI       16064
PIRM_REG_DATA_LT       10
GALIOJIMO_LAIK          0
APSKRITIS              29
dtype: int64

In [14]:
# Trinam nereikalingą stulpelį
df = df.drop(columns=['GALIOJIMO_LAIK'])
df = df.drop(columns=['GAMYBOS_METAI'])

In [15]:
# Visos eilutės su NaN reikšmėmis 
eilutes_trinimui_nan = df[df.isnull().any(axis=1)]
eilutes_trinimui_nan

,MARKE,KOMERCINIS_PAV,ELEKTRINE_TP,PIRM_REG_DATA_LT,APSKRITIS
171,NISSAN,LEAF 24KWH,Y,2020-11-26,NaN
1050,VOLKSWAGEN. VW,ID.3 PRO S 150 KW,Y,NaN,KLA
1719,E-MAX,NaN,Y,2024-06-22,KAU
1819,TESLA,MODEL X,Y,2024-05-15,NaN
1904,MINI,COOPER SE,Y,2024-06-01,NaN
2468,HYUNDAI,IONIQ,Y,2021-10-21,NaN
3913,E-MAX,NaN,Y,2024-03-04,MAR
4979,SKODA,ENYAQ 80,Y,2022-05-27,NaN
5318,AUDI,E-TRON 55,Y,2022-12-16,NaN
5711,JAGUAR,JAGUAR I-PACE,Y,2024-06-25,NaN


Ištrinam Nan reikšmes iš dataframe

In [16]:
# Trinam eilutes su NaN reikšmėmis
df = df.dropna()
df

,MARKE,KOMERCINIS_PAV,ELEKTRINE_TP,PIRM_REG_DATA_LT,APSKRITIS
0,NISSAN,LEAF,Y,2017-04-29,KAU
1,TESLA MOTORS,MODEL X,Y,2017-05-19,KAU
2,Nuasmeninta,Nuasmeninta,Y,2017-07-10,KAU
3,NISSAN,LEAF,Y,2017-09-08,TEL
4,NISSAN,NISSAN LEAF 40KWH,Y,2018-11-30,KAU
...,...,...,...,...,...
16065,MAZDA,MAZDA MX-30,Y,2024-04-04,KAU
16066,MAZDA,MAZDA MX-30,Y,2024-04-11,VIL
16067,GREENMO,-,Y,2024-04-23,KAU
16068,GREENMO,-,Y,2024-04-27,KAU


In [17]:
# Pakartotinis tikrinimas
df.isnull().any()

MARKE               False
KOMERCINIS_PAV      False
ELEKTRINE_TP        False
PIRM_REG_DATA_LT    False
APSKRITIS           False
dtype: bool

### Trinam eilutes kur stulpelyje 'KOMERCINIS_PAV' yra reikšmė 'Nuasmeninta' (susije su duomenų apsauga)

In [18]:
# Eilučių kiekis
df['KOMERCINIS_PAV'].str.contains('Nuasmeninta').sum()

665

In [19]:
# Eilučių peržiūra prieš trinimą
eilutes_trinimui = df[df['KOMERCINIS_PAV'] == 'Nuasmeninta']
eilutes_trinimui

,MARKE,KOMERCINIS_PAV,ELEKTRINE_TP,PIRM_REG_DATA_LT,APSKRITIS
2,Nuasmeninta,Nuasmeninta,Y,2017-07-10,KAU
11,Nuasmeninta,Nuasmeninta,Y,2020-04-07,PAN
22,Nuasmeninta,Nuasmeninta,Y,2020-08-01,KAU
25,Nuasmeninta,Nuasmeninta,Y,2020-12-23,TAU
29,Nuasmeninta,Nuasmeninta,Y,2020-12-11,VIL
...,...,...,...,...,...
14670,Nuasmeninta,Nuasmeninta,Y,2021-06-25,ALY
14688,Nuasmeninta,Nuasmeninta,Y,2021-09-18,VIL
15735,Nuasmeninta,Nuasmeninta,Y,2021-06-04,ALY
15748,Nuasmeninta,Nuasmeninta,Y,2021-06-30,ŠIA


In [20]:
# Trinam nereikalingas eilutes
df = df.drop(eilutes_trinimui.index)
df

,MARKE,KOMERCINIS_PAV,ELEKTRINE_TP,PIRM_REG_DATA_LT,APSKRITIS
0,NISSAN,LEAF,Y,2017-04-29,KAU
1,TESLA MOTORS,MODEL X,Y,2017-05-19,KAU
3,NISSAN,LEAF,Y,2017-09-08,TEL
4,NISSAN,NISSAN LEAF 40KWH,Y,2018-11-30,KAU
5,NISSAN,LEAF,Y,2019-06-05,PAN
...,...,...,...,...,...
16065,MAZDA,MAZDA MX-30,Y,2024-04-04,KAU
16066,MAZDA,MAZDA MX-30,Y,2024-04-11,VIL
16067,GREENMO,-,Y,2024-04-23,KAU
16068,GREENMO,-,Y,2024-04-27,KAU


In [21]:
# Triname visas nereikalingas eilutes funkcijos pagalba
df = eiluciu_trinimas('ELECTRIC SCOOTER', 'KOMERCINIS_PAV', df)
df = eiluciu_trinimas('-', 'KOMERCINIS_PAV', df)
df

,MARKE,KOMERCINIS_PAV,ELEKTRINE_TP,PIRM_REG_DATA_LT,APSKRITIS
0,NISSAN,LEAF,Y,2017-04-29,KAU
1,TESLA MOTORS,MODEL X,Y,2017-05-19,KAU
3,NISSAN,LEAF,Y,2017-09-08,TEL
4,NISSAN,NISSAN LEAF 40KWH,Y,2018-11-30,KAU
5,NISSAN,LEAF,Y,2019-06-05,PAN
...,...,...,...,...,...
16063,E-MAX,VT-120LD,Y,2024-03-30,KAU
16064,MERCEDES-BENZ,EQC 400 4MATIC,Y,2024-06-14,KAU
16065,MAZDA,MAZDA MX-30,Y,2024-04-04,KAU
16066,MAZDA,MAZDA MX-30,Y,2024-04-11,VIL


In [22]:
# suindeksuojam iš naujo
df = df.reset_index(drop=True)
df

,MARKE,KOMERCINIS_PAV,ELEKTRINE_TP,PIRM_REG_DATA_LT,APSKRITIS
0,NISSAN,LEAF,Y,2017-04-29,KAU
1,TESLA MOTORS,MODEL X,Y,2017-05-19,KAU
2,NISSAN,LEAF,Y,2017-09-08,TEL
3,NISSAN,NISSAN LEAF 40KWH,Y,2018-11-30,KAU
4,NISSAN,LEAF,Y,2019-06-05,PAN
...,...,...,...,...,...
13786,E-MAX,VT-120LD,Y,2024-03-30,KAU
13787,MERCEDES-BENZ,EQC 400 4MATIC,Y,2024-06-14,KAU
13788,MAZDA,MAZDA MX-30,Y,2024-04-04,KAU
13789,MAZDA,MAZDA MX-30,Y,2024-04-11,VIL


In [23]:
#konvertuojame stulpelį 'PIRM_REG_METAI_LT' į datetime formatą
df['PIRM_REG_DATA_LT'] = pd.to_datetime(df['PIRM_REG_DATA_LT'])

# pridedu naują stulpelį
df['PIRM_REG_METAI_LT'] = df['PIRM_REG_DATA_LT'].dt.strftime('%Y-%m')

# Trinam nereikalingą stulpelį
df = df.drop(columns=['PIRM_REG_DATA_LT'])
df

,MARKE,KOMERCINIS_PAV,ELEKTRINE_TP,APSKRITIS,PIRM_REG_METAI_LT
0,NISSAN,LEAF,Y,KAU,2017-04
1,TESLA MOTORS,MODEL X,Y,KAU,2017-05
2,NISSAN,LEAF,Y,TEL,2017-09
3,NISSAN,NISSAN LEAF 40KWH,Y,KAU,2018-11
4,NISSAN,LEAF,Y,PAN,2019-06
...,...,...,...,...,...
13786,E-MAX,VT-120LD,Y,KAU,2024-03
13787,MERCEDES-BENZ,EQC 400 4MATIC,Y,KAU,2024-06
13788,MAZDA,MAZDA MX-30,Y,KAU,2024-04
13789,MAZDA,MAZDA MX-30,Y,VIL,2024-04


In [24]:
# Apskričių kodai
print(df['APSKRITIS'].unique())

['KAU' 'TEL' 'PAN' 'VIL' 'TAU' 'KLA' 'MAR' 'ŠIA' 'UTE' 'ALY']


In [25]:
# Pakeičiame į suprantamus pavadinimus
pakeit_apskritis = {
    'VIL': 'Vilniaus',
    'KAU': 'Kauno',
    'KLA': 'Klaipėdos',
    'PAN': 'Panevėžio',
    'TEL': 'Telšių',
    'TAU': 'Tauragės',
    'MAR': 'Marijampolės',
    'ŠIA': 'Šiaulių',
    'UTE': 'Utenos',
    'ALY': 'Alytaus'
}
df = df.replace(pakeit_apskritis)
df

,MARKE,KOMERCINIS_PAV,ELEKTRINE_TP,APSKRITIS,PIRM_REG_METAI_LT
0,NISSAN,LEAF,Y,Kauno,2017-04
1,TESLA MOTORS,MODEL X,Y,Kauno,2017-05
2,NISSAN,LEAF,Y,Telšių,2017-09
3,NISSAN,NISSAN LEAF 40KWH,Y,Kauno,2018-11
4,NISSAN,LEAF,Y,Panevėžio,2019-06
...,...,...,...,...,...
13786,E-MAX,VT-120LD,Y,Kauno,2024-03
13787,MERCEDES-BENZ,EQC 400 4MATIC,Y,Kauno,2024-06
13788,MAZDA,MAZDA MX-30,Y,Kauno,2024-04
13789,MAZDA,MAZDA MX-30,Y,Vilniaus,2024-04


In [26]:
# Įrašome į failą sutvarkytus duomenis
df.to_csv('data_ev.csv', index=False)